**Step 1: Import Libraries**

In [102]:
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

**Step 2: Set Up the Dash Application**

In [103]:
# Initialize the Dash app
app = dash.Dash(__name__)

**Step 3: Load the data**

In [104]:
df = pd.read_csv('titanic.csv')
print(df.head())

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


**Step 4: Prepare Data**

In [105]:
# Drop rows with missing 'Embarked' values
df = df.dropna(subset=['Embarked'])

# Convert 'Sex' and 'Embarked' columns to categorical for filtering
df['Sex'] = df['Sex'].astype('category')
df['Embarked'] = df['Embarked'].astype('category')

# Select relevant columns for visualization
df = df[['Survived', 'Pclass', 'Sex', 'Age', 'Fare', 'Embarked']]

# Preview the cleaned dataset
print(df.head())

   Survived  Pclass     Sex   Age     Fare Embarked
0         0       3    male  22.0   7.2500        S
1         1       1  female  38.0  71.2833        C
2         1       3  female  26.0   7.9250        S
3         1       1  female  35.0  53.1000        S
4         0       3    male  35.0   8.0500        S


**Step 5: Create the Layout for the Dashboard**

In [106]:
# Create the layout for the dashboard
app.layout = html.Div([
    # Header
    html.H1("Titanic Data Dashboard", style={'textAlign': 'center'}),

    # Description
    html.Div("Use the controls below to explore Titanic data interactively.", style={'textAlign': 'center'}),

    # Dropdown for selecting a passenger class
    html.Div([
        html.Label("Select Passenger Class:"),
        dcc.Dropdown(
            id='pclass-dropdown',
            options=[
                {'label': 'Class 1', 'value': 1},
                {'label': 'Class 2', 'value': 2},
                {'label': 'Class 3', 'value': 3}
            ],
            value=1  # Default value
        ),
    ], style={'width': '50%'}),

    # Slider for selecting an age range
    html.Div([
        html.Label("Select Age Range:"),
        dcc.RangeSlider(
            id='age-slider',
            min=df['Age'].min(),
            max=df['Age'].max(),
            step=1,
            marks={int(age): str(int(age)) for age in range(int(df['Age'].min()), int(df['Age'].max())+1, 10)},
            value=[df['Age'].min(), df['Age'].max()]  # Default range
        )
    ], style={'width': '50%'}),

    # Graphs
    html.Div([
        dcc.Graph(id='line-chart'),  # Line chart placeholder
        dcc.Graph(id='scatter-plot')  # Scatter plot placeholder
    ])
])


**Step 6: Create the Callback Functions for Interactivity**

In [107]:
@app.callback(
    [Output('line-chart', 'figure'),
     Output('scatter-plot', 'figure')],
    [Input('pclass-dropdown', 'value'),
     Input('age-slider', 'value')]
)
def update_graphs(selected_pclass, age_range):
    # Filter the dataset based on selected class and age range
    filtered_df = df[
        (df['Pclass'] == selected_pclass) & 
        (df['Age'] >= age_range[0]) & 
        (df['Age'] <= age_range[1])
    ]

    # Handle empty dataset
    if filtered_df.empty:
        no_data_figure = px.line(
            title="No Data Available",
            labels={'Survived': 'Survival Rate', 'Age': 'Age Group'}
        )
        return no_data_figure, no_data_figure

    # Ensure numeric data for Age and Fare
    filtered_df['Age'] = pd.to_numeric(filtered_df['Age'], errors='coerce')
    filtered_df['Fare'] = pd.to_numeric(filtered_df['Fare'], errors='coerce')

    # --- Line Chart: Survival Rate by Age Group ---
    age_groups = pd.cut(filtered_df['Age'], bins=range(0, 81, 10))  # Group by decades
    survival_rate_by_age = (
        filtered_df.groupby(age_groups)['Survived'].mean().reset_index()
    )

    # Convert intervals to strings for serialization
    survival_rate_by_age['Age'] = survival_rate_by_age['Age'].astype(str)

    line_chart = px.line(
        survival_rate_by_age,
        x='Age',
        y='Survived',
        title='Survival Rate by Age Group',
        labels={'Survived': 'Survival Rate', 'Age': 'Age Group'}
    )

    # --- Scatter Plot: Age vs Fare ---
    scatter_plot = px.scatter(
        filtered_df,
        x='Age',
        y='Fare',
        color='Survived',
        title='Age vs. Fare',
        labels={'Fare': 'Fare', 'Age': 'Age', 'Survived': 'Survival Status'},
        hover_data=['Pclass', 'Sex', 'Embarked']
    )

    return line_chart, scatter_plot


**Step 7: Run the App**

In [108]:
if __name__ == '__main__':
    app.run_server(debug=True, port=8060)


C:\Users\moussad\AppData\Local\Temp\ipykernel_15116\564867561.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\moussad\AppData\Local\Temp\ipykernel_15116\564867561.py:25: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\moussad\AppData\Local\Temp\ipykernel_15116\564867561.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org